In [1]:
import numpy as np

# read sample data
with open('data/day08_sample.txt') as fh:
    data = [line.strip() for line in fh.readlines()]

# build as matrix
m = np.asarray([[int(x) for x in group] for group in data])
print(m)

# count border: just in case we don't have a n x n matrix
border_count = m.shape[0] + m.shape[1] + 2 * (m[:,0].shape[0] - 2)
print(border_count)

[[3 0 3 7 3]
 [2 5 5 1 2]
 [6 5 3 3 2]
 [3 3 5 4 9]
 [3 5 3 9 0]]
16


In [2]:
def quad_scan(r,c, m):
    """Take input (r,c) and determine if tree can be seen from any angle"""
    v = m[r,c]
    
    # each check is boolean - confirm all elements in direction are less than
    dirs = [np.all(v > m[r,:c]),
            np.all(v > m[r,c+1:]),
            np.all(v > m[:r, c]),
            np.all(v > m[r+1:, c])
           ]
    
    return np.any([d for d in dirs])
    
assert(quad_scan(1,1,m) == True) # top left
assert(quad_scan(1,3,m) == False) # top right
assert(quad_scan(2,1,m) == True) # left middle

row,col = m.shape
tot_vis = np.sum([quad_scan(r,c,m) for r in range(1,row-1) for c in range(1,col-1)])
        
print(f"Part 1 Visibile trees: {tot_vis + border_count}")

Part 1 Visibile trees: 21


In [3]:
# Actual Data
with open('data/day08.txt') as fh:
    data = [line.strip() for line in fh.readlines()]

m = np.asarray([[int(x) for x in group] for group in data])

# count border: just in case we don't have a n x n matrix
border_count = m.shape[0] + m.shape[1] + 2 * (m[:,0].shape[0] - 2)
print(border_count)

row,col = m.shape
tot_vis = np.sum([quad_scan(r,c,m) for r in range(1,row-1) for c in range(1,col-1)])

print(f"Part 1 Visibile trees: {tot_vis + border_count}")

392
Part 1 Visibile trees: 1684


In [4]:
# Part 2: Scenic Score
def scen_score(r,c, m):
    """Take input (r,c) and determine scenic score
    
    The general logic is based on indexing:
    - Looking left find the max index where the value of our tree <= another tree. 
      If this index was 0 (1st) and our tree was index 3 (4th) then we want a value of 3 - 0 = 3
    - Looking right find the min index where the value of our tree <= another tree. 
      If this index was 8 (9th tree) and our tree was index 3 (4th) then we want a value of 5;
      This is handled by treating tress 4 (0 index) - 8 (4th index) as a separate array. Can then do 
      min of this array where condition is met (which is 4th index) and add 1 to get 5 (account for 0-based index)
    
    Same logic applies for up/down. Up we reference starting row, down treat as separate vector
    
    Unfortunately I was running into issues handling null vectors (the case where every tree is less in a direction)
    meaning that we get an empty vector. The quick solve was to use try/except for each direction. However, I have
    to imagine there is a smarter way to do this (maybe checking length before subtracting )
    """
    v = m[r,c]
    
    # left
    try:
        l = c - np.max(np.where(v <= m[r,:c]))
    except:
        l = c
    
    # right
    try:
        ri = 1 + np.min(np.where(v <= m[r,c+1:]))
    except:
        ri = m.shape[0] - (c  + 1 )              
    
    # up
    try:
        u = r - np.max(np.where(v <= m[:r,c]))
    except:
        u = r   
        
    # down
    try:
        d = 1 + np.min(np.where(v <= m[r+1:,c]))
    except:
        d = m.shape[1] - (r + 1)  

    return l * ri * u * d

In [5]:
# read sample data
with open('data/day08_sample.txt') as fh:
    data = [line.strip() for line in fh.readlines()]

m = np.asarray([[int(x) for x in group] for group in data])

assert(scen_score(3,2,m) == 8)
assert(scen_score(1,2,m) == 4)

row,col = m.shape
max_scen_score = np.max([scen_score(r,c,m) for r in range(1,row-1) for c in range(1,col-1)])

print(f"Part 2 sample: max is {max_scen_score}")

Part 2 sample: max is 8


In [6]:
# exaple of error: all trees to left of the first 5 are visible (just a vector of [2])
r = 1
c = 1
v = m[r,c]
print(m[r, :])
try:
    l = c - np.max(np.where(v <= m[r,:c]))
except:
    l = c
print(l)

[2 5 5 1 2]
1


In [7]:
# read real data
with open('data/day08.txt') as fh:
    data = [line.strip() for line in fh.readlines()]

# build matrix
m = np.asarray([[int(x) for x in group] for group in data])
row,col = m.shape
max_scen_score = np.max([scen_score(r,c,m) for r in range(1,row-1) for c in range(1,col-1)])
print(f"Part 2: max is {max_scen_score}")


Part 2: max is 486540
